## Microsoft Kavram Çizgesi

[Microsoft Kavram Çizgesi](https://concept.research.microsoft.com/), kavramlar arasında `is-a` (bir örneğidir) ilişkileri olan, internetten çıkarılan geniş bir terimler sınıflandırmasıdır.

Bağlam Grafiği iki biçimde mevcuttur:
  * İndirmek için büyük metin dosyası
  * REST API'si

İstatistik:
 * 5401933 eşsiz kavram, 
 * 12551613 eşsiz örnek,
 * 87603947 `is-a` ilişkisi

## Web Hizmetini Kullanma

Web hizmeti, farklı gruplara ait bir kavramın olasılığını tahmin etmek için farklı çağrılar sunar. Daha fazla bilgiyi [burada](https://concept.research.microsoft.com/Home/Api) bulabilirsiniz.
İşte aranacak örnek URL: `https://concept.research.microsoft.com/api/Concept/ScoreByProb?instance=microsoft&topK=10`

In [ ]:
import urllib
import json
import ssl

def http(x):
    ssl._create_default_https_context = ssl._create_unverified_context
    response = urllib.request.urlopen(x)
    data = response.read()
    return data.decode('utf-8')

def query(x):
    return json.loads(http("https://concept.research.microsoft.com/api/Concept/ScoreByProb?instance={}&topK=10".format(urllib.parse.quote(x))))

query('microsoft')

Haber başlıklarını ana kavramlarını kullanarak kategorize etmeye çalışalım. Haber başlıklarını almak için [NewsApi.org](http://newsapi.org) hizmetini kullanacağız. Hizmeti kullanmak için kendi API anahtarınızı almanız gerekir - web sitesine gidin ve ücretsiz geliştirici planına kaydolun.

In [ ]:
newsapi_key = '<your API key here>'
def get_news(country='us'):
    res = json.loads(http("https://newsapi.org/v2/top-headlines?country={0}&apiKey={1}".format(country,newsapi_key)))
    return res['articles']

all_titles = [x['title'] for x in get_news('us')+get_news('gb')]

In [ ]:
all_titles

Her şeyden önce, haber başlıklarından isimleri çıkarabilmek istiyoruz. Bunu yapmak için bunun gibi birçok olağan DDİ görevini basitleştiren `TextBlob` kütüphanesini kullanacağız.

In [ ]:
import sys
!{sys.executable} -m pip install textblob
!{sys.executable} -m textblob.download_corpora
from textblob import TextBlob

In [ ]:
w = {}
for x in all_titles:
    for n in TextBlob(x).noun_phrases:
        if n in w:
            w[n].append(x)
        else:
            w[n]=[x]
{ x:len(w[x]) for x in w.keys()}

İsimlerin bize geniş konu ile ilgili gruplar vermediğini görebiliriz. İsimleri kavram çizgesinden elde edilen daha genel terimlerle değiştirelim. Bu biraz zaman alacak çünkü her bir isim öbeği için REST çağrısı yapıyoruz.

In [ ]:
w = {}
for x in all_titles:
    for noun in TextBlob(x).noun_phrases:
        terms = query(noun.replace(' ','%20'))
        for term in [u for u in terms.keys() if terms[u]>0.1]:
            if term in w:
                w[term].append(x)
            else:
                w[term]=[x]

In [ ]:
{ x:len(w[x]) for x in w.keys() if len(w[x])>3}

In [ ]:
print('\nECONOMY:\n'+'\n'.join(w['economy']))
print('\nNATION:\n'+'\n'.join(w['nation']))
print('\nPERSON:\n'+'\n'.join(w['person']))